# Preparing the dataset for hippocampus segmentation

In this notebook you will use the skills and methods that we have talked about during our EDA Lesson to prepare the hippocampus dataset using Python. Follow the Notebook, writing snippets of code where directed so using Task comments, similar to the one below, which expects you to put the proper imports in place. Write your code directly in the cell with TASK comment. Feel free to add cells as you see fit, but please make sure that code that performs that tasked activity sits in the same cell as the Task comment.


In [1]:
# TASK: Import the following libraries that we will use: nibabel, matplotlib, numpy
import nibabel as nib
import os
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import itertools

It will help your understanding of the data a lot if you were able to use a tool that allows you to view NIFTI volumes, like [3D Slicer](https://www.slicer.org/). I will refer to Slicer throughout this Notebook and will be pasting some images showing what your output might look like.

## Loading NIFTI images using NiBabel

NiBabel is a python library for working with neuro-imaging formats (including NIFTI) that we have used in some of the exercises throughout the course. Our volumes and labels are in NIFTI format, so we will use nibabel to load and inspect them.

NiBabel documentation could be found here: https://nipy.org/nibabel/

Our dataset sits in two directories - *images* and *labels*. Each image is represented by a single file (we are fortunate to have our data converted to NIFTI) and has a corresponding label file which is named the same as the image file.

Note that our dataset is "dirty". There are a few images and labels that are not quite right. They should be quite obvious to notice, though. The dataset contains an equal amount of "correct" volumes and corresponding labels, and you don't need to alter values of any samples in order to get the clean dataset.

In [2]:
# TASK: Your data sits in directory /data/TrainingSet.
# Load an image and a segmentation mask into variables called image and label
image = nib.load("/Users/taruntalreja/Desktop/data/TrainingSet/images/hippocampus_001.nii.gz")
print(image.shape)
label = nib.load("/Users/taruntalreja/Desktop/data/TrainingSet/labels/hippocampus_001.nii.gz")
print(label.shape)

(35, 51, 35)
(35, 51, 35)


In [3]:
# Nibabel can present your image data as a Numpy array by calling the method get_fdata()
# The array will contain a multi-dimensional Numpy array with numerical values representing voxel intensities. 
# In our case, images and labels are 3-dimensional, so get_fdata will return a 3-dimensional array. You can verify this
# by accessing the .shape attribute. What are the dimensions of the input arrays?

# TASK: using matplotlib, visualize a few slices from the dataset, along with their labels. 
# You can adjust plot sizes like so if you find them too small:
# plt.rcParams["figure.figsize"] = (10,10)
def show_slices(slices):
	""" Function to display row of image slices """
	fig, axes = plt.subplots(1, len(slices))
	for i, slice in enumerate(slices):
		axes[i].imshow(slice.T, cmap="gray", origin="lower")

slice_a = label.get_fdata()[26, :, :]
slice_b =  label.get_fdata()[:, 30, :]
slice_c =  label.get_fdata()[:, :, 16]
show_slices([slice_a, slice_b, slice_c])
plt.suptitle("Label Slices")  
plt.show()

In [4]:
slice_a = image.get_fdata()[26, :, :]
slice_b =  image.get_fdata()[:, 30, :]
slice_c =  image.get_fdata()[:, :, 16]
show_slices([slice_a, slice_b, slice_c])
plt.suptitle("Image Slices")  
plt.show()


Load volume into 3D Slicer to validate that your visualization is correct and get a feel for the shape of structures.Try to get a visualization like the one below (hint: while Slicer documentation is not particularly great, there are plenty of YouTube videos available! Just look it up on YouTube if you are not sure how to do something)

![3D slicer](img/Slicer.png)

In [5]:
img=mpimg.imread('/Users/taruntalreja/Desktop/Alzheimer-progression/img/Slicer.png')
imgplot = plt.imshow(img)
plt.show()

## Looking at single image data
In this section we will look closer at the NIFTI representation of our volumes. In order to measure the physical volume of hippocampi, we need to understand the relationship between the sizes of our voxels and the physical world.

In [6]:
# Nibabel supports many imaging formats, NIFTI being just one of them. I told you that our images 
# are in NIFTI, but you should confirm if this is indeed the format that we are dealing with
# TASK: using .header_class attribute - what is the format of our images?
# Slicer Render slicer image is based on below youtube video: 
# https://www.youtube.com/watch?v=MKLWzD0PiIc
# img = mpimg.imread('slicer_render.png')
# imgplot = plt.imshow(img)
# plt.show()
print(image.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : 
db_name         : 
extents         : 0
session_error   : 0
regular         : r
dim_info        : 0
dim             : [ 3 35 51 35  1  1  1  1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [1. 1. 1. 1. 1. 0. 0. 0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : 5.0.10
aux_file        : none
qform_code      : scanner
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : 1.0
qoffset_y       : 1.0
qoffset_z       : 1.0
srow_x          : [1. 0. 0. 1.]
srow_y          : [0. 1. 0. 1.]
srow_

Further down we will be inspecting .header attribute that provides access to NIFTI metadata. You can use this resource as a reference for various fields: https://brainder.org/2012/09/23/the-nifti-file-format/

In [7]:
# TASK: How many bits per pixel are used?
# 8

# TASK: What are the units of measurement?
# xyzt_units

# TASK: Do we have a regular grid? What are grid spacings?
# The grid spacings is as follows:   
# [1. 1. 1. 1. 1. 0. 0. 0.]

# TASK: What dimensions represent axial, sagittal, and coronal slices? How do you know?
# Affine attribute gives us information about the affine
# Affine: array-like or None, optional. Affine transform for the data. 
# This is used to determine how the data should be sliced for plotting into the 
# sagittal, coronal, and axial view axes. If None, identity is assumed. 
# The aspect ratio of the data are inferred from the affine transform.



In [8]:
np.set_printoptions(precision=3, suppress=True)

In [12]:
# By now you should have enough information to decide what are dimensions of a single voxel
# TASK: Compute the volume (in mm³) of a hippocampus using one of the labels you've loaded. 
# You should get a number between ~2200 and ~4500
all_volumes=list()
anterior_volumes=list()
posterior_volumes=list()

volume = 0

sliced_labels = [[f,nib.load(os.path.join("/Users/taruntalreja/Desktop/data/TrainingSet/labels/", f))] 
                for f in os.listdir("/Users/taruntalreja/Desktop/data/TrainingSet/labels/")]

def getVolume(label):
    vol = 0
    for i in range(label.shape[0]):
        for j in range(label.shape[1]):
            for k in range(label.shape[2]):
                if label[i][j][k] != 0:
                    vol += 1
    return vol

def getAnteriorVolume(label):
    vol = 0
    for i in range(label.shape[0]):
        for j in range(label.shape[1]):
            for k in range(label.shape[2]):
                if label[i][j][k] == 1:
                    vol += 1
    return vol

def getPosteriorVolume(label):
    vol = 0
    for i in range(label.shape[0]):
        for j in range(label.shape[1]):
            for k in range(label.shape[2]):
                if label[i][j][k] == 2:
                    vol += 1
    return vol

for idx in range(260):
    t_label = sliced_labels[idx][1].get_fdata()
    vol = getVolume(t_label)
    all_volumes.append(vol)
    anterior_volumes.append(getAnteriorVolume(t_label))
    posterior_volumes.append(getPosteriorVolume(t_label))

nonoutlier_volumes = np.array([v for v in all_volumes if (v<4500 and v>2200)])
anterior_volumes = np.array([v for v in anterior_volumes if (v<4500 and v>2200)])
posterior_volumes = np.array([v for v in posterior_volumes if (v<4500 and v>2200)])
print(nonoutlier_volumes)
print(anterior_volumes)
print(posterior_volumes)


[4151 3536 2873 2382 3654 3222 3442 3095 3686 3455 2956 2618 2930 2753
 2451 3377 3724 3575 3113 3327 2863 2857 3994 3361 3208 3172 3445 3248
 3568 3814 3229 2920 2819 3000 3356 3869 3660 3946 3372 3918 3845 3847
 2697 3409 3423 3088 3831 3160 2714 3027 3478 3219 3000 3947 2943 3439
 3304 3516 3109 3997 3707 3165 2570 2535 2755 3823 2684 3785 3450 3661
 2588 3450 2870 3594 2948 3420 3435 3195 2613 3257 3127 3412 3479 3326
 2726 3292 3522 2546 3061 3983 3325 2980 3987 3623 2590 3349 2678 3371
 3216 2863 3856 3105 4071 2995 3773 3689 3335 3957 3671 3692 2593 3089
 2920 2634 3820 3742 2708 3582 3506 3534 3404 3146 3409 3353 3923 3456
 3220 2471 3658 3167 3252 3749 3600 3733 2945 3272 2629 3450 2416 2859
 3704 3622 2593 3718 4263 3092 3650 3023 3700 3285 3080 3519 3202 3628
 2714 3763 2950 2448 3340 4383 3374 3878 3150 3167 3605 2635 3431 2942
 2757 2704 3029 3979 3285 3102 3877 2665 3612 3575 3679 2890 2647 3194
 3210 2894 3502 4029 3581 3643 4401 3054 3333 3842 2830 2422 4074 2811
 3698 

## Plotting some charts

In [13]:
# TASK: Plot a histogram of all volumes that we have in our dataset and see how 
# our dataset measures against a slice of a normal population represented by the chart below.
plt.title('Histogram of Non outlier volumes')
plt.xlabel('Volume mm3')
plt.hist(nonoutlier_volumes, bins=10)
plt.show()

plt.title('Histogram of Anterior volumes')
plt.xlabel('Volume mm3')
plt.hist(nonoutlier_volumes, bins=10)
plt.show()

plt.title('Histogram of Posterior volumes')
plt.xlabel('Volume mm3')
plt.hist(nonoutlier_volumes, bins=10)
plt.show()

plt.title('Histogram of all volumes')
plt.xlabel('Volume mm3')
plt.hist(all_volumes, bins=10)
plt.show()



<img src="img/histogram1.png" width=400 align=left>

<img src="img/histogram2.png" width=400 align=left>

<img src="img/histogram3.png" width=400 align=left>


<img src="img/nomogram_fem_right.svg" width=400 align=left>

Do you see any outliers? Why do you think it's so (might be not immediately obvious, but it's always a good idea to inspect) outliers closer. If you haven't found the images that do not belong, the histogram may help you.

In the real world we would have precise information about the ages and conditions of our patients, and understanding how our dataset measures against population norm would be the integral part of clinical validation that we talked about in last lesson. Unfortunately, we do not have this information about this dataset, so we can only guess why it measures the way it is. If you would like to explore further, you can use the [calculator from HippoFit project](http://www.smanohar.com/biobank/calculator.html) to see how our dataset compares against different population slices

Did you notice anything odd about the label files? We hope you did! The mask seems to have two classes, labeled with values `1` and `2` respectively. If you visualized sagittal or axial views, you might have gotten a good guess of what those are. Class 1 is the anterior segment of the hippocampus and class 2 is the posterior one. 

For the purpose of volume calculation we do not care about the distinction, however we will still train our network to differentiate between these two classes and the background

## Final remarks

Congratulations! You have finished Section 1. 

In this section you have inspected a dataset of MRI scans and related segmentations, represented as NIFTI files. We have visualized some slices, and understood the layout of the data. We have inspected file headers to understand what how the image dimensions relate to the physical world and we have understood how to measure our volume. We have then inspected dataset for outliers, and have created a clean set that is ready for consumption by our ML algorithm. 

In the next section you will create training and testing pipelines for a UNet-based machine learning model, run and monitor the execution, and will produce test metrics. This will arm you with all you need to use the model in the clinical context and reason about its performance!